In [1]:
# Reload modules automatically
# https://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

In [10]:
import importlib
from src.graph_models import create_graph_model
import numpy as np
import torch
import exp_eval_robustness
import exp_train

In [26]:
importlib.reload(exp_eval_robustness)
importlib.reload(exp_train)
data_params = dict(
    graph_model = 'CSBM',
    classes = 2,
    n = 1000,
    n_per_class_trn = 400,
    K = 0.5,
    sigma = 1,
    avg_within_class_degree = 1.5 * 2,
    avg_between_class_degree = 0.5 * 2,
    inductive_samples = 1000
)

model_params = dict(
    label="GAT",
    model="GAT",
    dropout=0.,
    dropout_neighourhood=0.
)

train_params = dict(
    lr=1e-2,
    weight_decay=1e-3,
    patience=300,
    max_epochs=3000,
    inductive=True,
)

attack_params = dict(
    attack = "l2"
)

verbosity_params = dict(
    display_steps = 100,
    debug_lvl = "info"
)  

other_params = dict(
    device = 0,
    allow_tf32 = False,
    sacred_metrics = True
)

seed = 1

result = exp_train.run(data_params, model_params, train_params, verbosity_params, other_params, seed, None)
#result = exp_eval_robustness.run(data_params, model_params, train_params, attack_params, verbosity_params, other_params, seed, None)

2022-08-10 18:30:08 (INFO): Starting experiment exp_train with configuration:
2022-08-10 18:30:08 (INFO): data_params: {'graph_model': 'CSBM', 'classes': 2, 'n': 1000, 'n_per_class_trn': 400, 'K': 0.5, 'sigma': 1, 'avg_within_class_degree': 3.0, 'avg_between_class_degree': 1.0, 'inductive_samples': 1000}
2022-08-10 18:30:08 (INFO): model_params: {'label': 'GAT', 'model': 'GAT', 'dropout': 0.0, 'dropout_neighourhood': 0.0}
2022-08-10 18:30:08 (INFO): train_params: {'lr': 0.01, 'weight_decay': 0.001, 'patience': 300, 'max_epochs': 3000, 'inductive': True}
2022-08-10 18:30:08 (INFO): verbosity_params: {'display_steps': 100, 'debug_lvl': 'info'}
2022-08-10 18:30:08 (INFO): other_params: {'device': 0, 'allow_tf32': False, 'sacred_metrics': True}
2022-08-10 18:30:08 (INFO): seed: 1
2022-08-10 18:30:08 (INFO): db_collection: None
2022-08-10 18:30:08 (INFO): Currently on gpu device cuda:0
2022-08-10 18:30:08 (INFO): 
Epoch    0: loss_train: 0.73027, loss_val: 0.78791, acc_train: 0.49625, acc_v

In [9]:
data_params["sigma"] = 1
seed = 1

result = exp_eval_robustness.run(data_params, model_params, train_params, attack_params, verbosity_params, other_params, seed, None)

2022-08-10 14:11:26 (INFO): Starting experiment exp_eval_robustness with configuration:
2022-08-10 14:11:26 (INFO): data_params: {'graph_model': 'CSBM', 'classes': 2, 'n': 1000, 'n_per_class_trn': 400, 'K': 0.5, 'sigma': 1, 'avg_within_class_degree': 3.0, 'avg_between_class_degree': 1.0, 'inductive_samples': 1000}
2022-08-10 14:11:26 (INFO): model_params: {'label': 'GCN', 'model': 'DenseGCN', 'n_filters': 64}
2022-08-10 14:11:26 (INFO): train_params: {'lr': 0.01, 'weight_decay': 0.001, 'patience': 300, 'max_epochs': 3000, 'inductive': True}
2022-08-10 14:11:26 (INFO): attack_params: {'attack': 'l2'}
2022-08-10 14:11:26 (INFO): verbosity_params: {'display_steps': 100, 'debug_lvl': 'info'}
2022-08-10 14:11:26 (INFO): other_params: {'device': 0, 'allow_tf32': False, 'sacred_metrics': True}
2022-08-10 14:11:26 (INFO): seed: 1
2022-08-10 14:11:26 (INFO): db_collection: None
2022-08-10 14:11:26 (INFO): Currently on gpu device cuda:0
2022-08-10 14:11:27 (INFO): 
Epoch    0: loss_train: 0.7010

In [21]:
from src.data import calc_balanced_sample

In [22]:
class_counts = [10, 100, 100]
n_samples = 60
print(calc_balanced_sample(class_counts, n_samples))

[10 25 25]


In [35]:
from src.utils import accuracy

In [7]:
logits = torch.Tensor([[0.8, 0.2], [0.1, 0.9]])
labels = torch.Tensor([0, 1])
logits.argmax(1)[1]

tensor(1)